In [1]:
#Load module
import pandas as pd
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

unable to import 'smart_open.gcs', disabling that module


In [10]:
# read training file
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-22_TrainingData.csv'
dfTraining = pd.read_csv(filename)

In [35]:
#read testing file
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-22_TestingData.csv'
dfTesting = pd.read_csv(filename)

In [39]:
#save smaller testing file
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-24_SmallTestingData.csv'
dfTesting.iloc[:100].to_csv(filename, index = False)

In [12]:
#read final processed file
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-24_FinalResult.csv'
dfFinal = pd.read_csv(filename)

In [13]:
dfFinal.shape

(6000, 12)

In [31]:
#create dictionaries
IndexToReportID_Dict = dfTraining['RECORD_ID'].to_dict()
IndexToStrDesc_Dict = dfTraining['SUSPICION_DESC'].to_dict()
ReportIDToIndex_Dict = {j:i for i,j in IndexToReportID_Dict.items()}

## Model Training

In [36]:
#hyperparameters for neural network
maxEpoch = 5
vectorSize = 500
alpha = 0.025
minAlpha = alpha #no decreasing rate
minCount = 1

In [37]:
#Load module
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

#doc2Vec model
class similarDoc():
    def __init__(self, **kwargs):
        #predefine prameters
        self.Doc2VecModel = Doc2Vec(**kwargs)
        self.MaxEpoch = 10
        self.ModelFolder = 'D:/Users/figohjs/Documents/NLP/StrPrioritization/Model'
        self.ModelName = self.ModelFolder + '/' + datetime.now().strftime('%Y-%m-%d') + '_' + 'Doc2Vec.model'
        #indices are based on list of tagged document
        self.BadStrIndex = kwargs['bad_index']
        self.Threshold = kwargs['threshold']
        self.VectorSize = kwargs['vector_size']
        self.SimilarBadDocList = []
        
    def prepareText(self, ContentList):
        #tag document
#         self.TaggedDocList = [TaggedDocument(doc, tags = [str(no)]) 
#                               for no, doc in enumerate([i.split() for i in ContentList])]
        self.TaggedDocList = [TaggedDocument(doc[1], tags = [doc[0]]) 
                              for doc in zip(self.BadStrIndex, [i.split() for i in ContentList])]
    def trainModel(self):
        #build vocab
        self.Doc2VecModel.build_vocab(self.TaggedDocList)
        for epoch in range(self.MaxEpoch):
            print('Iteration - %s'%epoch)
            self.Doc2VecModel.train(self.TaggedDocList,
                                    total_examples = self.Doc2VecModel.corpus_count,
                                    epochs = epoch)
            #self.Doc2VecModel.alpha -= 0.002
        self.Doc2VecModel.save(self.ModelName)
        print('%s is saved'%self.ModelName)
        
    def loadModel(self, TrainedModel):
        self.Doc2VecModel = gensim.models.doc2vec.Doc2Vec.load(TrainedModel)
                    
    def findSimilarDocForBadDoc(self, NewDoc, n = 10):
        NewVec = self.Doc2VecModel.infer_vector(NewDoc.split())
        similarRepList = []
        similarScoreList = []
        #loop through every bad report
        for index in self.BadStrIndex:
            similarScore = cosine_similarity(self.Doc2VecModel.docvecs[index].reshape(1, self.VectorSize),
                                             NewVec.reshape(1, self.VectorSize))
            #if similarity score exceeds threshold
            if similarScore >= self.Threshold:
                similarRepList.append(str(index))
                similarScoreList.append(str(similarScore[0][0]))
        #if not similar to any bad report, (a, b, c)
        #a - 1 if has similar to any bad report
        #b - list of reports 
        if len(similarRepList) == 0:
            return (0, '', '')
        else:
            return (1, ','.join(similarRepList), ','.join(similarScoreList))

## Model Testing

In [38]:
#hyperparameters for neural network
maxEpoch = 5
vectorSize = 500
alpha = 0.025
minAlpha = alpha #no decreasing rate
minCount = 1
badIndex = list(dfTraining[dfTraining['TP']].index)
#instantiate a class named similarDoc
paramDict = {'min_count': minCount, 'vector_size': vectorSize,
             'alpha':alpha, 'min_alpha':alpha, 'bad_index':badIndex,
            'threshold':0.75}

nlpSimilarity = similarDoc(**paramDict)

In [39]:
import time

start = time.time()
#load model
trainedModel = '2020-03-19_Doc2Vec.model'
nlpSimilarity.loadModel(trainedModel)
print(f'Time taken: {round(time.time() - start, 2)} seconds')

Time taken: 0.58 seconds


In [41]:
start = time.time()
#use trained model to find if there is report similar with any bad reports
similarList = []
indexList = []
scoreList = []
for no in range(dfTesting.shape[0]):
    info = str(dfTesting['SUSPICION_DESC_CLEAN'].values[no])
    similarBool, index, score = nlpSimilarity.findSimilarDocForBadDoc(info)
    similarList.append(similarBool)
    indexList.append(index)
    scoreList.append(score)
pandaDict = {'similarBool':similarList,
            'reportIndexList':indexList,
            'scoreList':scoreList}
dfResult = pd.DataFrame(pandaDict)
dfResult = pd.concat([dfResult, dfTesting.reset_index()], axis = 1)
dfResult.fillna('', inplace = True)
print(f'Time: {time.time() - start}')

Time: 1258.3744292259216


In [42]:
#get most similar report
def mostSimilarReport(indexArray, scoreArray):
    #store result
    resultList = []
    
    #loop through every row
    for no, tempArray in enumerate(scoreArray):
        #get max score from every list splitted by tempArray
        tempScoreList = str(tempArray).split(',')
        tempIndexList = str(indexArray[no]).split(',')
        maxScore = max(tempScoreList)
        indexList = [(i, tempScoreList[no]) for no, i in enumerate(tempIndexList) if  tempScoreList[no] == maxScore]
        
        if len(indexList) > 1:
            #take first element for interim
            resultList.append([indexList[0][0], indexList[0][1]])
        elif len(indexList) == 1:
            resultList.append([indexList[0][0], indexList[0][1]])
        else:
            resultList.append(['', ''])
          
    return pd.DataFrame(resultList, columns = ['mostSimilarIndex', 'mostSimilarScore'])

In [43]:
dfResult2 = mostSimilarReport(dfResult['reportIndexList'].values, dfResult['scoreList'].values)

colList = ['RECORD_ID', 'ANALYST','SUSPICION_DESC', 'SUSPICION_DESC_CLEAN', 'TP', 'FN', 'FP', 'noFlag']
dfFinal = pd.concat([dfResult2, dfResult[colList].reset_index()], axis = 1).copy()

dfFinal['mostSimilarReportID'] = dfFinal['mostSimilarIndex'].map(lambda x:IndexToReportID_Dict[int(x)] 
                                                                 if str(x) not in ['', 'nan'] else '')

dfFinal['mostSimilarStrDesc'] = dfFinal['mostSimilarIndex'].map(lambda x:IndexToStrDesc_Dict[int(x)] 
                                                                 if str(x) not in ['', 'nan'] else '')

In [53]:
dfFinal['mostSimilarIndex'].fillna('', inplace = True)

In [54]:
#save final result - save time for dashboard purpose
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-27_FinalResult.csv'
dfFinal.to_csv(filename, index = False)

In [65]:
#success rate - flag how many FN
successNum = sum(np.logical_and((dfFinal['mostSimilarIndex']!='').values, dfFinal['FN'].values))
print("Success Rate : %s"%round((successNum * 100/sum(dfFinal['FN'])),2))

Success Rate : 4.83


## Streamlit script

In [36]:
%%writefile D:\Users\figohjs\Documents\NLP\StrPrioritization\Model\doc2VecModel.py
#Load module
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

#doc2Vec model
class similarDoc():
    def __init__(self, **kwargs):
        #predefine prameters
        self.Doc2VecModel = Doc2Vec(**kwargs)
        self.MaxEpoch = 10
        self.ModelFolder = 'D:/Users/figohjs/Documents/NLP/StrPrioritization/Model'
        self.ModelName = self.ModelFolder + '/' + datetime.now().strftime('%Y-%m-%d') + '_' + 'Doc2Vec.model'
        #indices are based on list of tagged document
        self.BadStrIndex = kwargs['bad_index']
        self.Threshold = kwargs['threshold']
        self.VectorSize = kwargs['vector_size']
        self.SimilarBadDocList = []
        
    def prepareText(self, ContentList):
        #tag document
#         self.TaggedDocList = [TaggedDocument(doc, tags = [str(no)]) 
#                               for no, doc in enumerate([i.split() for i in ContentList])]
        self.TaggedDocList = [TaggedDocument(doc[1], tags = [doc[0]]) 
                              for doc in zip(self.BadStrIndex, [i.split() for i in ContentList])]
    def trainModel(self):
        #build vocab
        self.Doc2VecModel.build_vocab(self.TaggedDocList)
        for epoch in range(self.MaxEpoch):
            print('Iteration - %s'%epoch)
            self.Doc2VecModel.train(self.TaggedDocList,
                                    total_examples = self.Doc2VecModel.corpus_count,
                                    epochs = epoch)
            #self.Doc2VecModel.alpha -= 0.002
        self.Doc2VecModel.save(self.ModelName)
        print('%s is saved'%self.ModelName)
        
    def loadModel(self, TrainedModel):
        self.Doc2VecModel = gensim.models.doc2vec.Doc2Vec.load(TrainedModel)
                    
    def findSimilarDocForBadDoc(self, NewDoc, n = 10):
        NewVec = self.Doc2VecModel.infer_vector(NewDoc.split())
        similarRepList = []
        similarScoreList = []
        #loop through every bad report
        for index in self.BadStrIndex:
            similarScore = cosine_similarity(self.Doc2VecModel.docvecs[index].reshape(1, self.VectorSize),
                                             NewVec.reshape(1, self.VectorSize))
            #if similarity score exceeds threshold
            if similarScore >= self.Threshold:
                similarRepList.append(str(index))
                similarScoreList.append(str(similarScore[0][0]))
        #if not similar to any bad report, (a, b, c)
        #a - 1 if has similar to any bad report
        #b - list of reports 
        if len(similarRepList) == 0:
            return (0, '', '')
        else:
            return (1, ','.join(similarRepList), ','.join(similarScoreList))

Overwriting D:\Users\figohjs\Documents\NLP\StrPrioritization\Model\doc2VecModel.py


In [14]:
recordIdList = ['123', '234', '456']
feedbackList = ['', 'Review', 'No Review']
dfTemp = pd.DataFrame({'Record':recordIdList, 'Feedback':feedbackList})
dfTemp.head()

,Record,Feedback
0,123,
1,234,Review
2,456,No Review


In [17]:
dictTemp = dfTemp.set_index('Record').to_dict()['Feedback']
dictTemp

{'123': '', '234': 'Review', '456': 'No Review'}

In [19]:
#feedback for recordId - 234
dictTemp['234']

'Review'

In [21]:
#loop through dict
for key, value in dictTemp.items():
    print('Record:%s' %key + ' and its feedback: %s'%value)

Record:123 and its feedback: 
Record:234 and its feedback: Review
Record:456 and its feedback: No Review


In [72]:
%%writefile D:\Users\figohjs\Documents\NLP\StrPrioritization\Streamlit\doc2Vec.py
#streamlit for training and testing

#import modules
#from one folder
import sys
modelFilePath = 'D:/Users/figohjs/Documents/NLP/StrPrioritization/Model'
sys.path.append(modelFilePath)
import doc2VecModel as docModel

#normal import
import streamlit as st
import pandas as pd
from datetime import datetime
import time
import numpy as np
import base64
import operator

#header
st.sidebar.header('Document Similarity')

#sidebars
option = st.sidebar.selectbox("Phase", ['Model Training', "Model Testing"])

#hyperparameters for neural network
maxEpoch = 5
vectorSize = 500
alpha = 0.025
minAlpha = alpha #no decreasing rate
minCount = 1

# read training file
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-22_TrainingData.csv'
dfTraining = pd.read_csv(filename)

#read testing file
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-24_SmallTestingData.csv'
dfTesting = pd.read_csv(filename)

#define some parameters of the script
badIndex = list(dfTraining[dfTraining['TP']].index)
trainingData = dfTraining['SUSPICION_DESC_CLEAN'].values
trainButton = False
noUploadButton = False
dfFinal = pd.DataFrame()
trainedModel = 'D:/Users/figohjs/Documents/NLP/StrPrioritization/Model/2020-03-23_Doc2Vec.model'

#create dictionaries
IndexToReportID_Dict = dfTraining['RECORD_ID'].to_dict()
IndexToStrDesc_Dict = dfTraining['SUSPICION_DESC'].to_dict()
ReportIDToIndex_Dict = {j:i for i,j in IndexToReportID_Dict.items()}

#UDF
#get most similar report
def mostSimilarReport(indexArray, scoreArray):
    #store result
    resultList = []
    
    #loop through every row
    for no, tempArray in enumerate(scoreArray):
        #get max score from every list splitted by tempArray
        tempScoreList = str(tempArray).split(',')
        tempIndexList = str(indexArray[no]).split(',')
        maxScore = max(tempScoreList)
        indexList = [(i, tempScoreList[no]) for no, i in enumerate(tempIndexList) if  tempScoreList[no] == maxScore]
        
        if len(indexList) > 1:
            #take first element for interim
            resultList.append([indexList[0][0], indexList[0][1]])
        elif len(indexList) == 1:
            resultList.append([indexList[0][0], indexList[0][1]])
        else:
            resultList.append(['', ''])
          
    return pd.DataFrame(resultList, columns = ['mostSimilarIndex', 'mostSimilarScore'])

if option == 'Model Training':
    vectorSize = st.slider('Vector Size', 100, 1000, 500)
    alpha = st.slider('Learning Rate - Alpha', 0.025, 1., 0.025)
    threshold = st.slider('Threshold', 0.6, 1.0, 0.75)
    minCount = st.slider('Minimum frequency of a term', 1, 10, 1)
    trainButton = st.button('Train Model', key = 'train')
    
    #if trainButton is clicked
    if trainButton:
        st.markdown("Vector Size:%s"%vectorSize)
        st.markdown("Learning Rate - Alpha:%s"%alpha)
        st.markdown("Threshold:%s"%threshold)
        #instantiate a class named similarDoc
        paramDict = {'min_count': minCount, 'vector_size': vectorSize,
                     'alpha':alpha, 'min_alpha':alpha, 'bad_index':badIndex,
                    'threshold':threshold}

        nlpSimilarity = docModel.similarDoc(**paramDict)

        #training model
        nlpSimilarity.prepareText(trainingData)
        with st.spinner("Training a model"):
            nlpSimilarity.trainModel()
        st.success("Model is trained")
        st.markdown(datetime.now().strftime('%Y-%m-%d') + '_' + 'Doc2Vec.model is saved')
    
elif option == 'Model Testing':
    #upload file
    file = st.file_uploader("Upload file", type = ['csv'])
    noUploadButton = st.button('No Upload', key = 'upload')
    if noUploadButton:
        if trainButton:
            paramDict = {'min_count': minCount, 'vector_size': vectorSize,
                         'alpha':alpha, 'min_alpha':alpha, 'bad_index':badIndex,
                        'threshold':threshold}

            nlpSimilarity = docModel.similarDoc(**paramDict)
        else:
            maxEpoch = 5
            vectorSize = 500
            alpha = 0.025
            minAlpha = alpha #no decreasing rate
            minCount = 1
            threshold = 0.75
            paramDict = {'min_count': minCount, 'vector_size': vectorSize,
                 'alpha':alpha, 'min_alpha':alpha, 'bad_index':badIndex,
                'threshold':threshold}

            nlpSimilarity = docModel.similarDoc(**paramDict)

        #Displaying info from model we are using
        st.markdown("Vector Size:%s"%vectorSize)
        st.markdown("Learning Rate - Alpha:%s"%alpha)
        st.markdown("Threshold:%s"%threshold)

        #Loading a trained model
        with st.spinner("Loading a trained model"): 
            start = time.time()
            nlpSimilarity.loadModel(trainedModel)
        st.success("A trained model is loaded")
        st.markdown(f'Time taken: {round(time.time() - start, 2)} seconds')

        st.markdown('Finding similar bad Doc from testing data - %s rows'%dfTesting.shape[0])
        start = time.time()
        progressBar = st.progress(0)
        #use trained model to find if there is report similar with any bad reports
        similarList = []
        indexList = []
        scoreList = []
        for no in range(dfTesting.shape[0]):
            info = str(dfTesting['SUSPICION_DESC_CLEAN'].values[no])
            similarBool, index, score = nlpSimilarity.findSimilarDocForBadDoc(info)
            similarList.append(similarBool)
            indexList.append(index)
            scoreList.append(score)
            percent = int(no*100/dfTesting.shape[0])
            progressBar.progress(percent)
        st.markdown(f'Time taken: {round((time.time() - start)/60, 2)} minutes')

        #further data processing
        with st.spinner("Final data processing"):
            start = time.time()
            pandaDict = {'similarBool':similarList,
                        'reportIndexList':indexList,
                        'scoreList':scoreList}
            dfResult = pd.DataFrame(pandaDict)
            dfResult = pd.concat([dfResult, dfTesting.reset_index()], axis = 1)
            dfResult.fillna('', inplace = True)

            dfResult2 = mostSimilarReport(dfResult['reportIndexList'].values, dfResult['scoreList'].values)
            colList = ['RECORD_ID', 'ANALYST', 'SUSPICION_DESC_CLEAN', 'TP', 'FN', 'FP', 'noFlag']
            dfFinal = pd.concat([dfResult2, dfResult[colList].reset_index()], axis = 1).copy()

            dfFinal['mostSimilarReportID'] = dfFinal['mostSimilarIndex'].map(lambda x:IndexToReportID_Dict[int(x)] 
                                                                             if str(x) not in ['', 'nan'] else '')

            dfFinal['mostSimilarStrDesc'] = dfFinal['mostSimilarIndex'].map(lambda x:IndexToStrDesc_Dict[int(x)] 
                                                                             if str(x) not in ['', 'nan'] else '')
        st.success("Final data processing is completed")
        st.markdown(f'Time taken: {round(time.time() - start, 2)} seconds')
    
    if file:
        dfFinal = pd.read_csv(file)
    
    if dfFinal.shape[0]!=0:
        csvFile = dfFinal.to_csv(index = False)
        st.header("File Download - A Workaround for small data")
        b64 = base64.b64encode(csvFile.encode()).decode()  # some strings <-> bytes conversions necessary here
        href = f'<a href="data:file/csv;base64,{b64}">Download CSV File</a> (right-click and save as &lt;some_name&gt;.csv)'
        st.markdown(href, unsafe_allow_html=True)

        st.title('Evaluation')
        #calculate how many false negative
        successNum = sum(np.logical_and(pd.notnull(dfFinal['mostSimilarIndex']).values, dfFinal['FN'].values))
        successPercent = round((successNum * 100/sum(dfFinal['FN'])),2)
        st.markdown("Number of Reports is Flagged by Analyst (Rule does not flag):%s"%sum(dfFinal['FN']))
        st.markdown("Number of Reports is Flagged by Doc Similarity (Rule does not flag):%s"%successNum)
        st.markdown("Success Rate:%s percent"%successPercent)
        
        st.cache()
        st.title('Check results')
        badReportList = list(map(int, dfFinal[(dfFinal['FN']) & 
                                              (pd.notnull(dfFinal['mostSimilarIndex']))]['mostSimilarIndex'].unique()))
        badReportIDList = [IndexToReportID_Dict[i] for i in badReportList]
        reportID = st.selectbox('Bad Report ID', badReportIDList)
        reportInd = ReportIDToIndex_Dict[reportID]
        #prepare series to display
        series = pd.Series(IndexToStrDesc_Dict[reportInd])
        series.name = 'Description'
        table = pd.DataFrame().append(series)
        table.columns = ['']
        st.table(table)
        
        recordIDtoScoreDict = dict(sorted(dfFinal[dfFinal['mostSimilarIndex'].map(lambda x:x == float(reportInd))]\
                             .set_index('RECORD_ID')['mostSimilarScore']\
                             .to_dict().items(), key = operator.itemgetter(1), reverse = True))
        
        similarReportIDList = list(recordIDtoScoreDict.keys())
        similarReportID = st.selectbox('Similar Report ID', similarReportIDList)
        series = pd.Series(dfFinal[dfFinal['RECORD_ID'] == similarReportID]['SUSPICION_DESC'])
        series.name = 'Description'
        table = pd.DataFrame().append(series)
        table.columns = ['']
        st.write('Similarity Score: %s'%recordIDtoScoreDict[similarReportID])
        st.table(table)

Overwriting D:\Users\figohjs\Documents\NLP\StrPrioritization\Streamlit\doc2Vec.py


In [67]:
import operator
#  sorted(x.items(), key=operator.itemgetter(1))
recordIDtoScoreDict = sorted(dfFinal[dfFinal['mostSimilarIndex'].map(lambda x:x == '2')]\
                     .set_index('RECORD_ID')['mostSimilarScore']\
                     .to_dict().items(), key = operator.itemgetter(1), reverse = True)
dict(recordIDtoScoreDict)

{'117/SL/570603-X/S/2019/000652': '0.90683454',
 '117/SL/570603-X/S/2019/000524': '0.9054018',
 'DA/002/S/2019/003980': '0.90137756',
 '117/SL/570603-X/S/2019/000639': '0.8999478',
 '117/SL/570603-X/S/2019/000552': '0.8995643',
 'DA/002/S/2019/004603': '0.8989546',
 '117/SL/570603-X/S/2019/000605': '0.89864916',
 '117/SL/570603-X/S/2019/000574': '0.89546895',
 '117/SL/570603-X/S/2019/000740': '0.8940367',
 '117/SL/570603-X/S/2019/000584': '0.893069',
 '117/SL/570603-X/S/2019/000633': '0.8927595',
 '117/SL/570603-X/S/2019/000704': '0.8915929',
 '117/SL/570603-X/S/2019/000670': '0.89143133',
 '117/SL/570603-X/S/2019/000662': '0.889781',
 'AP/018/S/2019/001000': '0.8897016',
 'AP/018/S/2019/001015': '0.8896949',
 '117/SL/570603-X/S/2019/000734': '0.8885196',
 'AP/018/S/2019/000975': '0.887955',
 '117/SL/570603-X/S/2019/000694': '0.88775545',
 '117/SL/570603-X/S/2019/000540': '0.8865546',
 'AP/001/S/2019/000386': '0.88641894',
 '117/SL/570603-X/S/2019/000617': '0.88628036',
 'AP/018/S/2019

In [51]:
dfFinal['mostSimilarIndex'].unique()

array(['2', '', '417'], dtype=object)

In [ ]:
from IPython.display import display, HTML

display(HTML(chart_html))